In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchaudio
import torchaudio.transforms as transforms
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

### 1. Loading the data

In [ ]:
waveform, sample_rate = torchaudio.load(file_path, normalize=False)

print("Waveform shape:", waveform.shape)  # (channels, samples)
print("Sample rate:", sample_rate)

### 2. Transformation en Mel-spectrogramme

transform = transforms.MelSpectrogram(sample_rate)
mel_specgram = transform(waveform)  # (channel, n_mels, time)

In [ ]:
transform = transforms.AmplitudeToDB(stype="power", top_db=80)
mel_spect_db = transform(mel_specgram)
sns.heatmap()